In [1]:
import jieba
import pandas as pd
import gensim
import logging
import re
import os
import csv
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
%load_ext autoreload
%autoreload 2

In [2]:
textbook_path = '../ma_clean.txt'
#df = pd.read_csv(textbook_path, sep='\n', names=['documents'], header=None)

In [3]:
def get_corpus(fname):
    with open(fname) as f:
        lines = iter(f)
        #lines.next()
        for line in lines:
            yield line

class CorpusFriendly(object):
    def __init__(self,fname, tag=True, topK=None):
        self.topK = topK
        self.fname = fname
        self.tag = tag
    def __iter__(self):
        k = 1
        corpus_memory_friendly = get_corpus(self.fname)
        for context in corpus_memory_friendly:
            #data = gensim.utils.to_unicode(data).split(',')
            seg_context = jieba.cut(context, cut_all=True)
            words = " ".join(seg_context).split()
            #print ", ".join(words)
            label = [str(k)]
            if self.tag:
                yield gensim.models.doc2vec.TaggedDocument(words, label)
            else:
                yield words
            if self.topK:
                if k >= self.topK:
                    break
                k += 1

In [4]:
# corpus_train = CorpusFriendly(fname='hk_clean.txt',topK=3)
# for c in corpus_train:
#     print ", ".join(c.words)

In [5]:
# for c in get_corpus(textbook_path):
#     print c

## Create Corpus for Word2vec

In [289]:
import os.path
clean_sentences = ['../tw_clean_sentences.txt', '../hk_clean_sentences.txt', '../ma_clean_sentences.txt']
if reduce(lambda a, b: a*b,map(lambda x: os.path.exists(x), clean_sentences)):
     print "Files exisit!!"

Files exisit!!


In [61]:
text = []
for c in get_corpus(textbook_path):
    text.extend(c)

In [62]:
text_sentences = []
for t in "".join(text).split('\n'):
    text_sentences.extend(t.split())

In [63]:
corpus_word2vec = []
for t in text_sentences:
    corpus_word2vec.extend(t.split('。'))
corpus_word2vec = filter(None, corpus_word2vec)

In [64]:
print len(corpus_word2vec)

2570


In [65]:
with open("../tw_clean_sentences.txt", 'wb') as outfile:
    writer = csv.writer(outfile, delimiter='\n')
    for t in corpus_word2vec:
        writer.writerow([t])

## Word2vec

In [134]:
num_features = 500
min_word_count = 3
num_workers = 4
context = 6
downsampling = 1e-3

In [351]:
corpus_train = CorpusFriendly(fname="../ma_clean_sentences.txt", tag=False)
model_word2vec = gensim.models.word2vec.Word2Vec(corpus_train, 
                                        size=num_features, 
                                        min_count=min_word_count, 
                                        iter=30, 
                                        window=context, 
                                        workers=4)
                                        #sample=downsampling)

2017-01-02 15:41:25,814 : INFO : collecting all words and their counts
2017-01-02 15:41:25,815 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-01-02 15:41:26,154 : INFO : collected 12907 word types from a corpus of 96272 raw words and 4010 sentences
2017-01-02 15:41:26,170 : INFO : min_count=3 retains 4679 unique words (drops 8228)
2017-01-02 15:41:26,171 : INFO : min_count leaves 85850 word corpus (89% of original 96272)
2017-01-02 15:41:26,184 : INFO : deleting the raw counts dictionary of 12907 items
2017-01-02 15:41:26,185 : INFO : sample=0.001 downsamples 35 most-common words
2017-01-02 15:41:26,186 : INFO : downsampling leaves estimated 74380 word corpus (86.6% of prior 85850)
2017-01-02 15:41:26,186 : INFO : estimated required memory for 4679 words and 500 dimensions: 21055500 bytes
2017-01-02 15:41:26,197 : INFO : resetting layer weights
2017-01-02 15:41:26,276 : INFO : training model with 4 workers on 4679 vocabulary and 500 features, using sg=0

In [269]:
result_word2vec = model_word2vec.most_similar(positive=[u'毛泽东'], topn=5)
for e in result_word2vec:
    print e[0], e[1]

2017-01-01 23:59:50,528 : INFO : precomputing L2-norms of word weight vectors


展开 0.770020604134
毛 0.752099812031
党 0.747189760208
东路 0.743687391281
党内 0.733431816101


## Doc2Vec

In [245]:
# corpus_train = CorpusFriendly(fname=textbook_path)
# model_doc2vec = gensim.models.doc2vec.Doc2Vec(size=2000, min_count=1, iter=20, window=10, workers=6)
# model_doc2vec.build_vocab(corpus_train)

In [246]:
# model_doc2vec.train(corpus_train)

In [110]:
#model_doc2vec.save('models/gensim_doc2vec_hk_clean')

In [247]:
# result_doc2vec = model_doc2vec.most_similar(positive=[u'毛泽东'], topn=10)
# for e in result_doc2vec:
#     print e[0], e[1]

## Find top N adjective from similar words 

In [6]:
import jieba.posseg

### MostSimilarGenerator
MostSimilarGenerator is a generator to efficiently find all the adj and adv

In [7]:
class MostSimilarGenerator(object):
    def __init__(self, model, word=u'毛泽东'):
        self.word= word
        self.model= model
    def __iter__(self):
        for sim_word in self.model.most_similar(self.word, topn=len(self.model.vocab)):
            yield sim_word

### is_adj()
is_adj() will return True if input is an adjective, an adverb or an idiom, otherwise False

jieba.posseg actually has ability to tag which part of speech is. I choose to tag adjective and adverb. Instead of using DictionaryServices, I find it makes more sense. However, we can always change the method.

In [145]:
def is_adj(word):
    word = jieba.posseg.cut(word)
    for w, f in word:
        if 'a' in f or 'i' in f:
            return True
    #print w, f
    return False

### find_topn_adj()
find_topn_adj() will return a list of most similar words and corresponding scores of the word of interest 

In [146]:
def find_topn_adj(n, model, word=u'毛泽东',):
    k = 0
    topn_adj = []
    topn_score = []
    for sim_word in MostSimilarGenerator(model, word=word):
        if is_adj(sim_word[0]):
            topn_adj.append(sim_word[0])
            topn_score.append(sim_word[1])
            k += 1
        if k >= n:
            #return zip(topn_adj, topn_score)
            return {'word':topn_adj, 'score':topn_score}

In [77]:
# 蒋介石（蒋中正） 毛泽东
adj_word2vec = []
topn_adj = find_topn_adj(n=20, model= model_word2vec, word=u'毛泽东')
for w, s in zip(topn_adj['word'], topn_adj['score']):
    print w, s
    adj_word2vec.append(w)
    if s < 0.5:
        break
print len(adj_word2vec)

乐观 0.64469063282
老 0.620627701283
亲密 0.620467662811
面红 0.602802634239
最高 0.583639144897
整肃 0.576191544533
正确 0.547226250172
公开 0.541342735291
大党 0.539614021778
言者无罪 0.530300855637
牛鬼蛇神 0.506935596466
独秀 0.499838531017
12


In [248]:
# adj_doc2vec = []
# for w, s in find_topn_adj(n=200, model= model_doc2vec, word=u'毛泽东'):
#     print w, s
#     adj_doc2vec.append(w)

In [11]:
import re, math, collections, itertools
import nltk, nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist, ConditionalFreqDist

In [299]:
pos_files = ['../sentiment_dicts/正面情感词语（中文）.txt', 
             '../sentiment_dicts/正面评价词语（中文）.txt']
             #'../sentiment_dicts/NTUSD_positive_simplified.txt']
pos_words = []
for p in pos_files:
    with open(p) as posFile:
        words = posFile.readlines()
        words = list(map(lambda x:x.strip(),words))
        #print len(posWords)
        pos_words.extend(words)
print len(pos_words)

4570


In [300]:
neg_files = ['../sentiment_dicts/负面情感词语（中文）.txt', 
             '../sentiment_dicts/负面评价词语（中文）.txt']
             #'../sentiment_dicts/NTUSD_negative_simplified.txt']
neg_words = []
for n in neg_files:
    with open(n) as negFile:
        words = negFile.readlines()
        words = list(map(lambda x:x.strip(),words))
        #print len(posWords)
        neg_words.extend(words)
print len(neg_words)

4374


In [301]:
def pn_composition(wordlist):
    composition = {'pos':[], 'neg':[], 'unknown':[]}
    for r in wordlist:
        if r.encode('utf-8') in pos_words:
            composition['pos'].append(r)
        elif r.encode('utf-8') in neg_words:
            composition['neg'].append(r)
        else:
            composition['unknown'].append(r)
    return composition
    

In [302]:
def pos_neg_ratio(wordlist):
    composition = {'pos':[], 'neg':[], 'unknown':[]}
    for r in wordlist:
        if r.encode('utf-8') in pos_words:
            composition['pos'].append(r)
        elif r.encode('utf-8') in neg_words:
            composition['neg'].append(r)
        else:
            composition['unknown'].append(r)
    try:
        return len(composition['pos']) / float(len(composition['neg']))
    except ZeroDivisionError:
        return len(composition['pos'])

In [352]:
word = u'蒋介石'
#word = u'蒋中正'
adj_word2vec = find_topn_adj(n=20, model= model_word2vec, word= word)['word']
score_word2vec = find_topn_adj(n=20, model= model_word2vec, word=word)['score']
for a,s in zip(adj_word2vec, score_word2vec):
    print a, s
print pos_neg_ratio(adj_word2vec)
# for p in pn_composition(adj_word2vec)['neg']:
#     print p


2017-01-02 15:41:38,638 : INFO : precomputing L2-norms of word weight vectors


复新 0.720214188099
北新 0.711165308952
全军覆没 0.70276850462
大举进攻 0.701167345047
九一八事 0.694559633732
正面 0.691871166229
南大 0.681910037994
反动势力 0.6732006073
壮烈 0.671368598938
钦差大臣 0.667848944664
连续 0.644109845161
公开 0.59958344698
恐怖 0.588566124439
黄 0.584655404091
轰轰烈烈 0.583952784538
恐慌 0.583698689938
举行会谈 0.582687854767
危急 0.577298045158
不久 0.570244193077
前仆后继 0.567859768867
1.0


In [346]:
word = u'毛泽东'
adj_word2vec = find_topn_adj(n=20, model= model_word2vec, word= word)['word']
score_word2vec = find_topn_adj(n=20, model= model_word2vec, word=word)['score']
for a,s in zip(adj_word2vec, score_word2vec):
    print a, s
print pos_neg_ratio(adj_word2vec)

老 0.590997099876
乐观 0.564539074898
亲密 0.544198811054
整肃 0.531194269657
最高 0.529323041439
独秀 0.517952919006
大中 0.51092684269
言者无罪 0.501269698143
面红 0.490559220314
公开 0.465627282858
基 0.461378455162
年轻 0.441419959068
旧 0.427567183971
具体 0.426327526569
正确 0.42267537117
束手无策 0.415853440762
不满情绪 0.408960044384
牛鬼蛇神 0.400862455368
宽松 0.3914026618
大势已去 0.387609213591
3.0


In [347]:
word = u'孙中山'
adj_word2vec = find_topn_adj(n=20, model= model_word2vec, word= word)['word']
score_word2vec = find_topn_adj(n=20, model= model_word2vec, word=word)['score']
for a,s in zip(adj_word2vec, score_word2vec):
    print a, s
print pos_neg_ratio(adj_word2vec)

大元 0.544372677803
薄弱 0.540436983109
成功 0.518168926239
容 0.5066857934
落实 0.487409770489
北新 0.487395226955
著名 0.441380292177
激愤 0.425225615501
不久 0.423867881298
独秀 0.413086593151
封建 0.400092869997
长 0.396668374538
大党 0.373572409153
确实 0.368044346571
束手无策 0.367626935244
刀枪不入 0.362145274878
不慎 0.35793274641
大势已去 0.354604721069
重新 0.352033913136
最高 0.338831812143
1.25


In [348]:
word = u'马克思'
adj_word2vec = find_topn_adj(n=20, model= model_word2vec, word= word)['word']
score_word2vec = find_topn_adj(n=20, model= model_word2vec, word=word)['score']
for a,s in zip(adj_word2vec, score_word2vec):
    print a, s
print pos_neg_ratio(adj_word2vec)

独秀 0.702444255352
广泛 0.673004627228
充满 0.621963143349
不良 0.604296207428
着重 0.594822227955
封建 0.592111170292
不适 0.590828537941
优越 0.586958408356
著名 0.575307369232
各阶 0.573041558266
偷工减料 0.565737724304
古 0.563042640686
轻重 0.557736873627
亲密 0.557665109634
乐观 0.554307103157
牛鬼蛇神 0.544725000858
中大 0.532047152519
恶劣 0.527855932713
言者无罪 0.52551484108
混乱 0.521520376205
0.6


In [349]:
word = u'国民党'
adj_word2vec = find_topn_adj(n=20, model= model_word2vec, word= word)['word']
score_word2vec = find_topn_adj(n=20, model= model_word2vec, word=word)['score']
for a,s in zip(adj_word2vec, score_word2vec):
    print a, s
print pos_neg_ratio(adj_word2vec)

束手无策 0.601246595383
容 0.517170786858
重建 0.495835900307
薄弱 0.479796588421
壮大 0.475466251373
健康 0.470888257027
精锐 0.467885673046
坚决 0.461356163025
强烈不满 0.455766826868
各阶 0.425942242146
持久 0.425493776798
强硬 0.412223219872
速战速决 0.403032541275
不利 0.397700428963
小 0.395010739565
强大 0.36047488451
大势已去 0.357838153839
彻底 0.356023609638
烈 0.34965634346
严格 0.348642766476
3.5


In [350]:
word = u'共产党'
adj_word2vec = find_topn_adj(n=20, model= model_word2vec, word= word)['word']
score_word2vec = find_topn_adj(n=20, model= model_word2vec, word=word)['score']
for a,s in zip(adj_word2vec, score_word2vec):
    print a, s
print pos_neg_ratio(adj_word2vec)

薄弱 0.56472837925
独秀 0.562929868698
坚决 0.519235610962
团结 0.503582775593
团结一致 0.465173721313
基 0.46359410882
最高 0.456141322851
大势已去 0.443681567907
容 0.440776407719
大党 0.436806231737
精锐 0.430891007185
落实 0.418730914593
公开 0.418326675892
不良 0.413984686136
一大 0.412838459015
积极参与 0.400486648083
束手无策 0.400304853916
不稳 0.396662384272
成功 0.396431148052
各阶 0.386137485504
3.0


In [343]:
# pn_ratio = []
# for num in range(0, len(adj_word2vec), 5):
#     adj_word2vec = find_topn_adj(n=num, model= model_word2vec, word=u'蒋介石')['word']
#     #print pos_neg_ratio(adj_word2vec)
#     pn_ratio.append(pos_neg_ratio(adj_word2vec))

In [18]:
composition = pn_composition(adj_word2vec)

In [280]:
#words_of_interest = [u'蒋介石', u'毛泽东', u'孙中山', u'国民党', u'共产党', u'马克思']
words_of_interest = [u'蒋中正', u'毛泽东', u'孙中山', u'国民党', u'共产党', u'马克思']

In [281]:
adj = {'center':{},'pos':{}, 'neg':{}, 'unknown':{}}
for woi in words_of_interest[:]:
    adj['center'][woi] = list(model_word2vec[woi])
    adj_word2vec = find_topn_adj(n=50, model= model_word2vec, word=woi)['word']
    composition = pn_composition(adj_word2vec)
    for word in composition['pos']:
        wordvec = list(model_word2vec[word])
        if word not in adj['pos']:
            adj['pos'][word] = wordvec
    for word in composition['neg']:
        wordvec = list(model_word2vec[word])
        if word not in adj['neg']:
            adj['neg'][word] = wordvec
    for word in composition['unknown']:
        wordvec = list(model_word2vec[word])
        if word not in adj['unknown']:
            adj['unknown'][word] = wordvec

In [282]:
with open('../tsne-word-embedding/tw.txt', 'wb') as outfile:
    writer = csv.writer(outfile, delimiter='\t')
    for w in adj['center']:
        writer.writerow([w.encode('utf-8')] + adj['center'][w])
    for w in adj['pos']:
        #print [w] + adj['pos'][w]
        writer.writerow([w.encode('utf-8')] + adj['pos'][w])
    for w in adj['neg']:
        writer.writerow([w.encode('utf-8')] + adj['neg'][w])
    for w in adj['unknown']:
        writer.writerow([w.encode('utf-8')] + adj['unknown'][w])
#     for w in model_word2vec.vocab.keys()[:2000]:
#         writer .writerow([w.encode('utf-8')] + list(model_word2vec[w]))